# Ratio
A class that implements fractional numbers. The `Ratio` class provides exact arithmetic for representing exact musical quantities such as proportional (metric) time, duration, and 'just' tuning intervals. Ratios can be created from two integers or from a string. Ratios are compared and combined using the standard math operators.

In [1]:
import math
from decimal import Decimal

class Ratio:
    pass

## `Ratio.__init__`
#### Creates a `Ratio` from integers, a floating point number, or a string name.
 * `Ratio(int, int)` - creates a ratio from an integer numerator and denominator.
 * `Ratio(int)` - creates a ratio from an integer numerator with the denominator set to 1.
 * `Ratio(float)` - creates a ratio from a floating point number (see: `as_integer_ratio()`)
 * `Ratio(string)` -  creates a ratio from a string 'num/den'. Both num and den must produce valid integers.

```
@param num If only num is specified it must be either an integer, float, or a string containing a valid ratio expression 'a/b'. If both num and den are provided they must both be integer value.
@param den If specified den must be a non-zero integer denominator
```

Upon construction the new ratio will always be expressed in its most simple form, for example `Ratio(6,12)` will become `Ratio(1/2)`, See: `gcd()`. If both the numerator and denominator are negative the ratio should be converted to positive by the constructor.

The constructor should raise a `TypeError` if the num or den is not an integer, string or float and a `ZeroDivisionError` if the denominator is 0.

In [2]:
def __init__(self, num, den=None):
    if isinstance(num, int):
        self.num = num
        
        if isinstance(den, int):
            self.den = den
        elif den == None:
            self.den = 1
        else:
            raise TypeError("Denominator is not an integer")
        
        if self.num * self.den > 0:
            self.num = abs(self.num)
            self.den = abs(self.den)
        else:
            self.num = -abs(self.num)
            self.den = abs(self.den)
    elif isinstance(num, float):
        if den != None:
            raise TypeError("Denominator should be None when numerator is float")
        self.num, self.den = Decimal(str(num)).as_integer_ratio()
    elif isinstance(num, str):
        length = len(num)
        self.num = ''
        self.den = ''
        i = 0
        has_slash = False
        
        if length > 0 and num[i] == '-':
            self.num += '-'
            i += 1
        
        while i < length:
            if num[i] >= '0' and num[i] <= '9':
                self.num += num[i]
                i += 1
            elif num[i] == '/':
                has_slash = True
                i += 1
                break
            else:
                raise TypeError("Numerator is not an integer")
        
        if has_slash:
            if length > 0 and num[i] == '-':
                self.num += '-'
                i += 1
            
            while i < length:
                if num[i] >= '0' and num[i] <= '9':
                    self.den += num[i]
                    i += 1
                else:
                    raise TypeError("Denominator is not an integer")
            
            try:
                self.num = int(self.num)
                self.den = int(self.den)
            except:
                raise ValueError("Denominator cannot be empty")
        else:
            raise ValueError("Denominator cannot be empty")
    
    if self.den == 0:
        raise ZeroDivisionError("Denominator is 0")
        
    if self.num * self.den > 0:
        self.num = abs(self.num)
        self.den = abs(self.den)
    else:
        self.num = -abs(self.num)
        self.den = abs(self.den)
    gcd = math.gcd(self.num, self.den)
    self.num, self.den = self.num // gcd, self.den // gcd

Ratio.__init__ = __init__

## `Ratio.__str__`
#### Returns a string showing the ratio's fraction and the `hex` value of the ratio's memory address.
Example: `<Ratio: 1/4 0x10610d2b0>`

In [3]:
def __str__(self):
    return f'<Ratio: {self.num}/{self.den} {hex(id(self))}>'

Ratio.__str__ = __str__

## `Ratio.__repr__`
#### Returns a string expression that will evaluate to this ratio.

In [4]:
def __repr__(self):
    return f'Ratio("{self.num}/{self.den}")'

Ratio.__repr__ = __repr__

## `Ratio.__mul__`
#### Implements `Ratio * Ratio`, `Ratio * int` and `Ratio * float`.

```
@param other An Ratio, int or float.
@returns A Ratio if other is a Ratio or an int, otherwise a float.
```
    
A `TypeError` should be raised if other is not a `Ratio`, `int` or `float`.

In [5]:
def __mul__(self, other):
    if isinstance(other, float):
        return self.num / self.den * other
    elif isinstance(other, Ratio):
        num, den = self.num * other.num, self.den * other.den
    elif isinstance(other, int):
        num, den = self.num * other, self.den
    else:
        raise TypeError("Other is not a Ratio, int or float")
    
    gcd = math.gcd(num, den)    
    return Ratio(num // gcd, den // gcd)

Ratio.__mul__ = __mul__

## `Ratio.__rmul__`
#### Implements right side multiplication by calling `__mul__`
`__rmul__ = __mul__`

In [6]:
def __rmul__(self, other):
    return __mul__(self, other)

Ratio.__rmul__ = __rmul__

## `Ratio.__invert__`
#### Implements 1 / ratio (reciprocal).

```
@returns A new Ratio.
```

In [7]:
def __invert__(self):
    if self.num == 0:
        raise ZeroDivisionError("Numerator is 0")
    elif self.num > 0:
        return Ratio(self.den, self.num)
    else:
        return Ratio(-self.den, -self.num)
        
Ratio.__invert__ = __invert__

## `Ratio.__truediv__`
#### Implements `Ratio / Ratio`, `Ratio / int` and `Ratio / float`.

```
@param other A Ratio, int or float.
@returns A Ratio if other is a Ratio or an int, otherwise a float.
```

A TypeError should be raised if other is not a `Ratio`, `int` or `float`.

In [8]:
def __truediv__(self, other):
    if isinstance(other, Ratio):
        return self * other.__invert__()
    elif isinstance(other, int):
        if other == 0:
            raise ZeroDivisionError("Other is 0")
        else:
            num, den = self.num, self.den * other

        gcd = math.gcd(num, den)
        return Ratio(num // gcd, den // gcd)
    elif isinstance(other, float):
        if other == 0:
            raise ZeroDivisionError("Other is 0")
        else:
            return self.num / self.den / other
    else:
        raise TypeError("Other is not a Ratio, int or float")
        
Ratio.__truediv__ = __truediv__

## `Ratio.__rtruediv__`
#### Implements `int / Ratio` or `float / Ratio` (right side division).

```
@returns A new Ratio.
```

In [9]:
def __rtruediv__(self, other):
    if isinstance(other, (Ratio, int, float)):
        return self.__invert__() * other
    else:
        raise TypeError("Other is not a Ratio, int or float")
        
Ratio.__rtruediv__ = __rtruediv__

## `Ratio.lcm`
#### A static method that returns the lowest common multiple of two integers a and b. lcm be calculated using `gcd()`: `(a * b) // gcd(a, b)`

In [10]:
def lcm(a, b):
    if isinstance(a, int) and isinstance(b, int):
        return (a * b) // math.gcd(a, b)
    else:
        raise TypeError("Input numbers are not int")
        
Ratio.lcm = lcm

## `Ratio.__add__`
#### Implements `Ratio + Ratio`, `Ratio + int` and `Ratio + float`.
In order to add two ratios their denominators must be converted to the least common multiple of the current denominator. See: `lcm()`.

```
@returns A new Ratio.
```

In [11]:
def __add__(self, other):
    if isinstance(other, Ratio):
        lcm = Ratio.lcm(self.den, other.den)
        return Ratio(self.num * lcm // self.den + other.num * lcm // other.den, lcm)
    elif isinstance(other, int):
        return Ratio(self.num + other * self.den, self.den)
    elif isinstance(other, float):
        return self.num / self.den + other
    else:
        raise TypeError("Other is not a Ratio, int or float")
        
Ratio.__add__ = __add__

## `Ratio.__radd__`
#### Implements right side addition by calling __add__.

```
@returns A new Ratio.
```
`__radd__ = __add__`

In [12]:
def __radd__(self, other):
    return self + other

Ratio.__radd__ = __radd__

## `Ratio.__neg__`
#### Implements `-ratio` (negation).
```
@returns A new Ratio.
```

In [13]:
def __neg__(self):
    return Ratio(self.num * -1, self.den)

Ratio.__neg__ = __neg__

## `Ratio.__sub__`
#### Implements `Ratio - Ratio`, `Ratio - int` and `Ratio - float`.

```
@returns A new Ratio.
```

In [14]:
def __sub__(self, other):
    return self + -other

Ratio.__sub__ = __sub__

## `Ratio.__rsub__`
#### Implements `int - Ratio` and `float - Ratio` (right side subtraction).

```
@returns A new Ratio.
```

In [15]:
def __rsub__(self, other):
    return -self + other

Ratio.__rsub__ = __rsub__

## `Ratio.__mod__`
#### Implements `Ratio % Ratio`.

```
@returns A new Ratio.
```

In [16]:
def __mod__(self, other):
    if not isinstance(other, (Ratio, int)):
        raise TypeError("Other is not a Ratio or int")
    elif isinstance(other, int):
        other = Ratio(other)
    
    q = int(self / other * 1.0)
    d = self - other * q
    
    if d.num < 0:
        if other.num < 0:
            d -= other
        else:
            d += other
    
    return d
    
Ratio.__mod__ = __mod__

## `Ratio.__pow__`
#### Implements `Ratio ** int`, `Ratio ** float`, and `Ratio ** Ratio`.

```
@returns If the exponent is a positive or negative `int` a `Ratio` should be returned. otherwise for `Ratio` or `float` exponents a `float` should be returned. See: `math.pow()`.
```

In [17]:
def __pow__(self, other):
    if isinstance(other, int):
        to_return = self
        
        if other < 0:
            to_return = to_return.__invert__()
            other = -other
            
        to_return.num = to_return.num ** other
        to_return.den = to_return.den ** other
        
        return to_return
    elif isinstance(other, (Ratio, float)):
        return (1.0 * self) ** (1.0 * other)
    else:
        raise TypeError("Other is not a Ratio, int or float")

Ratio.__pow__ = __pow__

## `Ratio.__rpow__`
#### Implements an `int ** Ratio` or `float ** Ratio`

```
@param other  The base integer or float.
@returns A floating point number.
```

The function can be implemented using math.pow().

In [18]:
def __rpow__(self, other):
    if isinstance(other, (int, float)):
        return (1.0 * other) ** (1.0 * self)
    else:
        raise TypeError("Other is not a int or float")

Ratio.__rpow__ = __rpow__

## `Ratio.__lt__`
#### Implements `Ratio < Ratio`, `Ratio < int`, `Ratio < float`. See: `compare()`.

In [19]:
def __lt__(self, other):
    if isinstance(other, (Ratio, int)):
        return (self - other).num < 0
    elif isinstance(other, float):
        return self - other < 0
    else:
        raise TypeError("Other is not a Ratio, int or float")
    
Ratio.__lt__ = __lt__

## `Ratio.__le__`
#### Implements `Ratio <= Ratio`, `Ratio <= int`, `Ratio <= float`. See: `compare()`.

In [20]:
def __le__(self, other):
    if isinstance(other, (Ratio, int)):
        return (self - other).num <= 0
    elif isinstance(other, float):
        return self - other <= 0
    else:
        raise TypeError("Other is not a Ratio, int or float")
    
Ratio.__le__ = __le__

## `Ratio.__eq__`
#### Implements `Ratio == Ratio`, `Ratio == int`, `Ratio == float`. See: `compare()`.

In [21]:
def __eq__(self, other):
    if isinstance(other, (Ratio, int)):
        return (self - other).num == 0
    elif isinstance(other, float):
        return self - other == 0
    else:
        return False
    
Ratio.__eq__ = __eq__

## `Ratio.__ne__`
#### Implements `Ratio != Ratio`, `Ratio != int`, `Ratio != float`. See: `compare()`.

In [22]:
def __ne__(self, other):
    return not self == other

Ratio.__ne__ = __ne__

## `Ratio.__ge__`
#### Implements `Ratio >= Ratio`, `Ratio >= int`, `Ratio >= float`. See: `compare()`.

In [23]:
def __ge__(self, other):
    if isinstance(other, (Ratio, int)):
        return (self - other).num >= 0
    elif isinstance(other, float):
        return self - other >= 0
    else:
        raise TypeError("Other is not a Ratio, int or float")
    
Ratio.__ge__ = __ge__

## `Ratio.__gt__`
#### Implements `Ratio > Ratio`, `Ratio > int`, `Ratio > float`. See: `compare()`.

In [24]:
def __gt__(self, other):
    if isinstance(other, (Ratio, int)):
        return (self - other).num > 0
    elif isinstance(other, float):
        return self - other > 0
    else:
        raise TypeError("Other is not a Ratio, int or float")
    
Ratio.__gt__ = __gt__

## `Ratio.__hash__`
#### Returns a single integer hash value for the ratio: `num << 16 + den`

In [25]:
def __hash__(self):
    return self.num << 16 + self.den

Ratio.__hash__ = __hash__

## `Ratio.compare`
#### Helper method implements ratio comparison.

Returns 0 if the ratios are equal, a negative value if self is less than other and a positive value if self is GEQ other. Given two ratios the comparison is `(num1 * den2) - (num2 * den1)`

In [26]:
def compare(self, other):
    if isinstance(other, (int, float)):
        other = Ratio(other)
        
    if not isinstance(other, Ratio):
        raise TypeError("Other is not a Ratio, int or float")
        
    return (self.num * other.den) - (other.num * self.den)

Ratio.compare = compare

## Ratio.string
#### Returns the string name of the ratio `'num/den'`.

In [27]:
def string(self):
    return f'{self.num}/{self.den}'

Ratio.string = string

## `Ratio.reciprocal`
#### Returns `1 / ratio`.

In [28]:
def reciprocal(self):
    return self.__invert__()

Ratio.reciprocal = reciprocal

## Ratio.dotted
#### Returns the musical 'dotted' value of the ratio, e.g. 1 / 4 with one dot is 1 / 4 + 1 / 8 = 3 / 8.

```
@param dots  The number of dots to apply, each dot adds half the previous value of the ratio.
@return A new ratio representing the dotted value.
```

The method should raise a `ValueError` if dots is not a positive integer.

In [29]:
def dotted(self, dots=1):
    if not isinstance(dots, int):
        raise ValueError("Dots is not a positive integer")
    elif dots <= 0:
        raise ValueError("Dots is not a positive integer")
    
    return self * (2 - Ratio(1, 2) ** dots)

Ratio.dotted = dotted

## `Ratio.tuplets`
#### Returns a list of num sub-divisions (metric 'tuples') that sum to value of `ratio * num`.

```
@param num  The number of tuples to return.
@param intimeof  A number that, when multiplied by the fraction itself, represents the sum of all the tuplets returned.
@returns A list of num ratios that sum to the value of the Ratio.
```

Examples: `Ratio(1,4).tuplets(3)` returns three tuplets `[1/12, 1/12, 1/12]` which sum to `Ratio(1,4)`.  `Ratio(1,4).tuplets(3,2)` returns threetuplets `[1/6, 1/6, 1/6]` which sum to `ratio * 2`, or `1/2`.

In [30]:
def tuplets(self, num, intimeof=1):
    if not isinstance(num, int):
        raise ValueError("Num is not a positive integer")
    elif num <= 0:
        raise ValueError("Num is not a positive integer")
    
    if not isinstance(intimeof, (Ratio, int)):
        raise TypeError("Intimeof is not a positive Ratio or int")
    elif intimeof <= 0:
        raise TypeError("Intimeof is not a positive Ratio or int")
        
    if isinstance(intimeof, int):
        intimeof = Ratio(intimeof)
    
    return [(self * intimeof / num) for i in range(0, num)]

Ratio.tuplets = tuplets

## `Ratio.tup`
#### Returns the ratio representing num divisions of this ratio.

```
@param num  The number to divide this ratio by.
@return The new tuple value ratio.
```

Example:  `Ratio(1,4).tup(5)` is `1/20`

In [31]:
def tup(self, num):
    if not isinstance(num, (Ratio, int)):
        raise TypeError("Num is not a positive Ratio or int")
    elif num <= 0:
        raise TypeError("Num is not a positive Ratio or int")
        
    if isinstance(num, int):
        num = Ratio(num)
        
    return self / num

Ratio.tup = tup

## `Ratio.float`
#### Returns the ratio as a floating point number.

In [32]:
def flt(self):
    return 1.0 * self

Ratio.flt = flt

## `Ratio.seconds`
#### Converts the ratio to floating point seconds according to a given tempo and beat:

```
@param tempo  The tempo in beats per minute. Defaults to 60.
@param beat  A ratio representing the beat. Defaults to 1/4 (quarter note).
```

In [33]:
def seconds(self, tempo=60, beat=Ratio(1, 4)):
    if not isinstance(tempo, (Ratio, int)):
        raise TypeError("Tempo is not a positive Ratio or int")
    elif tempo <= 0:
        raise TypeError("Tempo is not a positive Ratio or int")
        
    if isinstance(tempo, int):
        tempo = Ratio(tempo)
        
    if not isinstance(beat, (Ratio, int)):
        raise TypeError("Beat is not a positive Ratio or int")
    elif beat <= 0:
        raise TypeError("Beat is not a positive Ratio or int")
        
    if isinstance(beat, int):
        beat = Ratio(beat)
    
    return (self / beat / tempo * 60) * 1.0

Ratio.seconds = seconds